In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import scipy.io as sio
from sympy import symbols, solve
import math
from scipy.io import loadmat
import random
import cmath
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D






def pinknoise(DIM, BETA, MAG):
    """
    Generate 1/f spatial noise with a normal error distribution.

    Parameters:
    DIM (tuple): Size of the spatial pattern (e.g., DIM=(10, 5) for a 10x5 spatial grid)
    BETA (float): Spectral distribution parameter
                  BETA = 0 is random white noise
                  BETA = -1 is pink noise
                  BETA = -2 is Brownian noise
    MAG (float): Scaling variable for the noise amplitude

    Returns:
    np.ndarray: 2D array of pink noise
    """

    u = np.fft.fftfreq(DIM[0]).reshape(-1, 1)
    v = np.fft.fftfreq(DIM[1]).reshape(1, -1)

    S_f = (u**2 + v**2)
    S_f[S_f == 0] = 1  # Avoid division by zero
    S_f = S_f**(BETA/2.0)
    S_f[u**2 + v**2 == 0] = 0  # Set the zero frequency component back to zero

    phi = np.random.rand(*DIM)
    y = S_f**0.5 * (np.cos(2 * np.pi * phi) + 1j * np.sin(2 * np.pi * phi))
    y = y * MAG / np.max(np.abs(y))

    x = np.fft.ifft2(y)
    x = np.real(x)

    return x




def resting_state(mu2, mu1, nu, N):
    if N == 1:  # resting state on upper branch
        x_rs = mu2 / (3 * (mu1 / 2 + cmath.sqrt(mu1**2 / 4 - mu2**3 / 27))**(1/3)) + (mu1 / 2 + cmath.sqrt(mu1**2 / 4 - mu2**3 / 27))**(1/3)
    elif N == 2:  # resting state on lower branch
        x_rs = -mu2 / (6 * (mu1 / 2 + cmath.sqrt(mu1**2 / 4 - mu2**3 / 27))**(1/3)) - (mu1 / 2 + cmath.sqrt(mu1**2 / 4 - mu2**3 / 27))**(1/3) / 2 - (cmath.sqrt(3) * 1j * (mu2 / (3 * (mu1 / 2 + cmath.sqrt(mu1**2 / 4 - mu2**3 / 27))**(1/3)) - (mu1 / 2 + cmath.sqrt(mu1**2 / 4 - mu2**3 / 27))**(1/3))) / 2
    elif N == 3:
        x_rs = (cmath.sqrt(3) * 1j * (mu2 / (3 * (mu1 / 2 + cmath.sqrt(mu1**2 / 4 - mu2**3 / 27))**(1/3)) - (mu1 / 2 + cmath.sqrt(mu1**2 / 4 - mu2**3 / 27))**(1/3))) / 2 - (mu1 / 2 + cmath.sqrt(mu1**2 / 4 - mu2**3 / 27))**(1/3) / 2 - mu2 / (6 * (mu1 / 2 + cmath.sqrt(mu1**2 / 4 - mu2**3 / 27))**(1/3))
    else:
        x_rs = None  # handle unexpected N values

    return x_rs

def Random_bifurcation_path(bifurcation):
    curves = loadmat('curves.mat')

    if bifurcation == 1:
        onset_curve = curves['SNr_LCs']
        offset_curve = curves['SHl']

    elif bifurcation == 2:
        onset_curve = curves['SNr_LCb']
        offset_curve = curves['SHb']

    elif bifurcation == 3:
        onset_curve = curves['SNr_LCs']
        offset_curve = curves['SNl_ActiveRest']

    elif bifurcation == 4:
        onset_curve = curves['SNr_LCb']
        offset_curve = curves['FLC_top']

    elif bifurcation == 5:
        onset_curve = curves['SNr_ActiveRest']
        offset_curve = curves['SHl']

    elif bifurcation == 6:
        onset_curve = curves['SNr_ActiveRest']
        offset_curve = curves['SNl_ActiveRest']

    elif bifurcation == 7:
        onset_curve = curves['subH']
        offset_curve = curves['SHb']

    elif bifurcation == 8:
        onset_curve = curves['subH']
        offset_curve = curves['FLC']

    else:
        raise ValueError("Invalid bifurcation value")

    onset_curve_length = onset_curve.shape[1]
    offset_curve_length = offset_curve.shape[1]

    # One random path
    random_onset_index = random.randint(0, onset_curve_length - 1)
    random_offset_index = random.randint(0, offset_curve_length - 1)

    A = offset_curve[:, random_offset_index]
    B = onset_curve[:, random_onset_index]

    if random_offset_index < random_onset_index:
        A, B = B, A

    return A, B



def parametrization_2_points_arc(A, B, R):
    A = np.array(A)
    B = np.array(B)

    # Calculate E
    E = A / R

    # Calculate F
    F = np.cross(np.cross(A, B), A)
    F = F / np.linalg.norm(F)
    return E, F


def hysteresis_loop_model(t, x, k, R, dstar, E, F, N):
    mu2 = R * (E[0] * np.cos(x[2]) + F[0] * np.sin(x[2]))
    mu1 = -R * (E[1] * np.cos(x[2]) + F[1] * np.sin(x[2]))
    nu = R * (E[2] * np.cos(x[2]) + F[2] * np.sin(x[2]))

    x_rs = np.real(resting_state(mu2, mu1, nu, N))

    xdot = -x[1]
    ydot = x[0] ** 3 - mu2 * x[0] - mu1 - x[1] * (nu + x[0] + x[0] ** 2)
    zdot = -k * (np.sqrt((x[0] - x_rs) ** 2 + x[1] ** 2) - dstar)
    Xdot = np.array([xdot, ydot, zdot])
    return Xdot


# SETTINGS - INTEGRATION
x0 = np.array([0, 0, 0])  # initial conditions

# SETTINGS - MODEL
b = 1.0  # focus
R = 0.4  # radius
N = 1


CL = 6  # Choose index corresponding to class above
k = 0.003
dstar = 0.3
tstep = 0.01
tmax = 3000
tspan = np.arange(0, tmax + tstep, tstep)
A,B = Random_bifurcation_path(CL)
A = A.T
B = B.T
E, F = parametrization_2_points_arc(A, B, R)
N_t = len(tspan)
X = np.zeros((3, N_t))
xx = x0
sigma = 100

Rn1 = pinknoise((1, N_t), -1, sigma)
Rn2 = pinknoise((1, N_t), -1, 0)
Rn3 = pinknoise((1, N_t), -1, 0)

Rn = np.vstack([Rn1, Rn2, Rn3])


for n in range(N_t):
    # Euler-Meruyama method
    Fxx = hysteresis_loop_model(tspan[n], xx, k, R, dstar, E, F, N)
    xx = xx + tstep * Fxx + np.sqrt(tstep) * Rn[:, n]
    X[:, n] = xx



x = X.T
t = tspan

# Calculate Bursting Path
z = x[:,2]
mu2 = R * (E[0] * np.cos(z) + F[0] * np.sin(z))
mu1 = -R * (E[1] * np.cos(z) + F[1] * np.sin(z))
nu = R * (E[2] * np.cos(z) + F[2] * np.sin(z))


import plotly.graph_objects as go



# Load data
data_mesh = loadmat('testmesh.mat')
data_curves = loadmat('curves.mat')
data_curves_2 = loadmat('curves2.mat')
data_bif = loadmat('bifurcation_crossing.mat')
data_sphere = loadmat('sphere_mesh.mat')

BCSmesh = data_mesh['BCSmesh']
Active_restmesh = data_mesh['Active_restmesh']
Seizure_mesh = data_mesh['Seizure_mesh']
Bistable_Lcb_mesh = data_mesh['Bistable_Lcb_mesh']
Fold_of_cycles = data_bif['Fold_of_cycles']
Homoclinic_to_saddle3 = data_bif['Homoclinic_to_saddle3']
Homoclinic_to_saddle2 = data_bif['Homoclinic_to_saddle2']
Homoclinic_to_saddle1 = data_bif['Homoclinic_to_saddle1']
Homoclinic_to_saddle = data_bif['Homoclinic_to_saddle']
Fold = data_bif['Fold']
Hopf = data_bif['Hopf']
SNIC = data_curves_2['SNIC']
# Define sphere parameters
radius = 0.4
phi, theta = np.mgrid[0.0:np.pi:50j, 0.0:2.0*np.pi:50j]
# Parametric equations for the sphere
X_sphere = radius * np.sin(phi) * np.cos(theta)
Y_sphere = radius * np.sin(phi) * np.sin(theta)
Z_sphere = radius * np.cos(phi)




# Plotting
fig_timeseries = go.Figure()
fig_timeseries.add_trace(go.Scatter(x=t, y=x[:, 0], mode='lines', name='x', line=dict(color='black', width=1)))
fig_timeseries.update_layout(
    title='Timeseries',
    xaxis_title='t',
    yaxis_title='x',
    plot_bgcolor='white',  # Background color of the plot area
    paper_bgcolor='white'  # Background color of the whole figure
)

fig_timeseries.show()
# Create a Plotly figure
fig_mesh = go.Figure()

# BCSmesh
fig_mesh.add_trace(go.Mesh3d(
    x=BCSmesh['vertices'][0][0][:, 0],
    y=BCSmesh['vertices'][0][0][:, 1],
    z=BCSmesh['vertices'][0][0][:, 2],
    i=BCSmesh['faces'][0][0][:, 0] - 1,
    j=BCSmesh['faces'][0][0][:, 1] - 1,
    k=BCSmesh['faces'][0][0][:, 2] - 1,
    opacity=0.3,
    color='rgba(248, 246, 184, 0.3)',
    name='BCSmesh'
))

# Active_restmesh
fig_mesh.add_trace(go.Mesh3d(
    x=Active_restmesh['vertices'][0][0][:, 0],
    y=Active_restmesh['vertices'][0][0][:, 1],
    z=Active_restmesh['vertices'][0][0][:, 2],
    i=Active_restmesh['faces'][0][0][:, 0] - 1,
    j=Active_restmesh['faces'][0][0][:, 1] - 1,
    k=Active_restmesh['faces'][0][0][:, 2] - 1,
    opacity=0.3,
    color='rgba(235, 235, 235, 0.7)',
    name='Active_restmesh'
))

# Seizure_mesh
fig_mesh.add_trace(go.Mesh3d(
    x=Seizure_mesh['vertices'][0][0][:, 0],
    y=Seizure_mesh['vertices'][0][0][:, 1],
    z=Seizure_mesh['vertices'][0][0][:, 2],
    i=Seizure_mesh['faces'][0][0][:, 0] - 1,
    j=Seizure_mesh['faces'][0][0][:, 1] - 1,
    k=Seizure_mesh['faces'][0][0][:, 2] - 1,
    opacity=0.3,
    color='rgba(228, 180, 211, 0.2)',
    name='Seizure_mesh'
))

# Bistable_Lcb_mesh
fig_mesh.add_trace(go.Mesh3d(
    x=Bistable_Lcb_mesh['vertices'][0][0][:, 0],
    y=Bistable_Lcb_mesh['vertices'][0][0][:, 1],
    z=Bistable_Lcb_mesh['vertices'][0][0][:, 2],
    i=Bistable_Lcb_mesh['faces'][0][0][:, 0] - 1,
    j=Bistable_Lcb_mesh['faces'][0][0][:, 1] - 1,
    k=Bistable_Lcb_mesh['faces'][0][0][:, 2] - 1,
    opacity=0.3,
    color='rgba(248, 246, 184, 0.3)',
    name='Bistable_Lcb_mesh'
))




# Plotting curves
for scale in [1]:
    Fold_of_cycles_scaled = scale * Fold_of_cycles
    Homoclinic_to_saddle3_scaled = scale * Homoclinic_to_saddle3
    Homoclinic_to_saddle2_scaled = scale * Homoclinic_to_saddle2
    Homoclinic_to_saddle1_scaled = scale * Homoclinic_to_saddle1
    Homoclinic_to_saddle_scaled = scale * Homoclinic_to_saddle
    Fold_scaled = scale * Fold
    Hopf_scaled = scale * Hopf
    SNIC_scaled = scale * SNIC

    fig_mesh.add_trace(go.Scatter3d(
        x=Fold_of_cycles_scaled[0, :],
        y=Fold_of_cycles_scaled[1, :],
        z=Fold_of_cycles_scaled[2, :],
        mode='lines',
        line=dict(color='rgba(248, 68, 149, 1)', width=2),
        name='Fold of cycles (FLC)'
    ))

    fig_mesh.add_trace(go.Scatter3d(
        x=Homoclinic_to_saddle3_scaled[0, :],
        y=Homoclinic_to_saddle3_scaled[1, :],
        z=Homoclinic_to_saddle3_scaled[2, :],
        mode='lines',
        line=dict(color='rgba(103, 179, 217, 1)', width=2, dash='dash'),
        showlegend=False
    ))

    fig_mesh.add_trace(go.Scatter3d(
        x=Homoclinic_to_saddle2_scaled[0, :],
        y=Homoclinic_to_saddle2_scaled[1, :],
        z=Homoclinic_to_saddle2_scaled[2, :],
        mode='lines',
        line=dict(color='rgba(103, 179, 217, 1)', width=2),
        showlegend=False
    ))

    fig_mesh.add_trace(go.Scatter3d(
        x=Homoclinic_to_saddle1_scaled[0, :],
        y=Homoclinic_to_saddle1_scaled[1, :],
        z=Homoclinic_to_saddle1_scaled[2, :],
        mode='lines',
        line=dict(color='rgba(103, 179, 217, 1)', width=2, dash='dash'),
        showlegend=False
    ))

    fig_mesh.add_trace(go.Scatter3d(
        x=Homoclinic_to_saddle_scaled[0, :],
        y=Homoclinic_to_saddle_scaled[1, :],
        z=Homoclinic_to_saddle_scaled[2, :],
        mode='lines',
        line=dict(color='rgba(103, 179, 217, 1)', width=2),
        name='Saddle-Homoclinic (SH)'
    ))

    fig_mesh.add_trace(go.Scatter3d(
        x=Fold_scaled[0, 139:563],
        y=Fold_scaled[1, 139:563],
        z=Fold_scaled[2, 139:563],
        mode='lines',
        line=dict(color='rgba(244, 156, 52, 1)', width=2),
        name='Saddle Node (SN)'
    ))

    fig_mesh.add_trace(go.Scatter3d(
        x=Fold_scaled[0, 574:],
        y=Fold_scaled[1, 574:],
        z=Fold_scaled[2, 574:],
        mode='lines',
        line=dict(color='rgba(244, 156, 52, 1)', width=2),
       showlegend=False
    ))

    fig_mesh.add_trace(go.Scatter3d(
        x=Fold_scaled[0, :79],
        y=Fold_scaled[1, :79],
        z=Fold_scaled[2, :79],
        mode='lines',
        line=dict(color='rgba(244, 156, 52, 1)', width=2),
        showlegend=False
    ))

    fig_mesh.add_trace(go.Scatter3d(
        x=Fold_scaled[0, 564:574],
        y=Fold_scaled[1, 564:574],
        z=Fold_scaled[2, 564:574],
        mode='lines',
        line=dict(color='rgba(244, 156, 52, 1)', width=2, dash='dash'),
        showlegend=False
    ))

    fig_mesh.add_trace(go.Scatter3d(
        x=Hopf_scaled[0, 399:973],
        y=Hopf_scaled[1, 399:973],
        z=Hopf_scaled[2, 399:973],
        mode='lines',
        line=dict(color='rgba(116, 191, 69, 1)', width=2),
        name='Supercritical Hopf (SupH)'
    ))

    fig_mesh.add_trace(go.Scatter3d(
        x=Hopf_scaled[0, :400],
        y=Hopf_scaled[1, :400],
        z=Hopf_scaled[2, :400],
        mode='lines',
        line=dict(color='rgba(116, 191, 69, 1)', width=2, dash='dot'),
        name='Subcritical Hopf (SubH)'
    ))

    fig_mesh.add_trace(go.Scatter3d(
        x=SNIC_scaled[0, :],
        y=SNIC_scaled[1, :],
        z=SNIC_scaled[2, :],
        mode='lines',
        line=dict(color='rgba(244, 156, 52, 1)', width=2, dash='dash'),
        name='Saddle Node Invariant Cycle (SNIC)'
    ))
    # Add other traces similarly

# Plotting the sphere
fig_mesh.add_trace(go.Mesh3d(
    x=X_sphere.flatten(),
    y=Y_sphere.flatten(),
    z=Z_sphere.flatten(),
    opacity=0.3,
    color='rgba(76, 76, 76, 0.1)',
    name='Sphere',
    alphahull=0
))
fig_mesh.add_trace(go.Scatter3d(
        x=mu2,
        y=mu1,
        z=nu,
        mode='lines',
        line=dict(color='rgba(0,0,0, 1)', width=2),
        name='Bursting path'
    ))

fig_mesh.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        bgcolor='white',  # Background color of the 3D scene
    ),
    title='Parameter Space Plot',
    paper_bgcolor='white',  # Background color of the whole figure
    plot_bgcolor='white'  # Background color of the plot area
)

fig_mesh.show()

Buffered data was truncated after reaching the output size limit.